Before we train a model, we need to prepare the dataset.

First we download nearly 40MB examples of neutral and smiling faces.

Then we list all the image files, and create a big list of examples as `(path, class)` tuples.

In [1]:
import os
import fnmatch
from io import StringIO
import numpy as np
import PIL.Image
import IPython.display
import shutil
import math

In [2]:
def find_rectangle(n, max_ratio=2):
    sides = []
    square = int(math.sqrt(n))
    for w in range(square, max_ratio * square):
        h = n / w
        used = w * h
        leftover = n - used
        sides.append((leftover, (w, h)))
    return sorted(sides)[0][1]

# should work for 1d and 2d images, assumes images are square but can be overriden
def make_mosaic(images, n=None, nx=None, ny=None, w=None, h=None):
    if n is None and nx is None and ny is None:
        nx, ny = find_rectangle(len(images))
    else:
        nx = n if nx is None else nx
        ny = n if ny is None else ny
    images = np.array(images)
    if images.ndim == 2:
        side = int(np.sqrt(len(images[0])))
        h = side if h is None else h
        w = side if w is None else w
        images = images.reshape(-1, h, w)
    else:
        h = images.shape[1]
        w = images.shape[2]
    image_gen = iter(images)
    mosaic = np.empty((h*ny, w*nx))
    for i in range(ny):
        ia = (i)*h
        ib = (i+1)*h
        for j in range(nx):
            ja = j*w
            jb = (j+1)*w
            mosaic[ia:ib, ja:jb] = next(image_gen)
    return mosaic

In [3]:
def show_array(a, fmt='png', filename=None):
    a = np.squeeze(a)
    a = np.uint8(np.clip(a, 0, 255))
    image_data = StringIO()
    IPython.display.display(IPython.display.Image(data=image_data.getvalue()))


In [4]:
def list_all_files(directory, extensions=None):
    for root, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            base, ext = os.path.splitext(filename)
            joined = os.path.join(root, filename)
            if extensions is None or ext.lower() in extensions:
                yield joined

In [6]:
negative_paths = list(list_all_files('SMILEsmileD-master/SMILEs/negatives/negatives7/', ['.jpg']))
print('loaded', len(negative_paths), 'negative examples')
positive_paths = list(list_all_files('SMILEsmileD-master/SMILEs/positives/positives7/', ['.jpg']))
print('loaded', len(positive_paths), 'positive examples')
examples = [(path, 0) for path in negative_paths[:3690]] + [(path, 1) for path in positive_paths]
print(len(examples))

loaded 9475 negative examples
loaded 3690 positive examples
7380


For each path, we will load the image with `imread` and resize from 64x64 to 32x32 with `block_reduce`. This can take up to **10 seconds**.

In [7]:
from skimage.measure import block_reduce
from skimage.io import imread

def examples_to_dataset(examples, block_size=2):
    X = []
    y = []
    for path, label in examples:
        img = imread(path, as_gray=True)
        img = block_reduce(img, block_size=(block_size, block_size), func=np.mean)
        X.append(img)
        y.append(label)
    return np.asarray(X), np.asarray(y)

%time X, y = examples_to_dataset(examples)

CPU times: user 16.1 s, sys: 2.32 s, total: 18.4 s
Wall time: 22 s


Then we turn `X` and `y` into numpy arrays and coerce them into the right shape and range.

In [8]:
X = X.astype(np.float32) / 255.
y = y.astype(np.int32)
print(X.dtype, X.min(), X.max(), X.shape)
print(y.dtype, y.min(), y.max(), y.shape)

float32 0.0 1.0 (7380, 32, 32)
int32 0 1 (7380,)


Let's take a look at a few of the images.

Now we save the data to disk for so we can load it quickly when working with it later. Keras likes to use data in the (rows, cols, channels) format so we make a small change to the images before saving them.

In [9]:
X = np.expand_dims(X, axis=-1)
np.save('X.npy', X)
np.save('y.npy', y)